In [1]:
import sys

sys.path.insert(0, '..')

import shutil
import os
import uuid
from biosimulator_processes.execute import exec_utc_comparison
from verification_service.worker.compare import Supervisor, Worker
from verification_service.data_model.shared import MongoDbConnector
from pymongo.mongo_client import MongoClient
from dotenv import load_dotenv
import os 

_outs = './test_outputs'
if os.path.exists(_outs):
    shutil.rmtree(_outs)


def jobid(): return str(uuid.uuid4())


load_dotenv("../verification_service/.env")
uri = os.getenv("MONGO_DB_URI")
omex_source_dir = './examples/sbml-core'
omex_name = 'Elowitz-Nature-2000-Repressilator.omex'
omex_fp = os.path.join(omex_source_dir, omex_name)
out_dir = './test_outputs'
simulators = ['amici', 'copasi', 'tellurium']
spec_name = 'cI mRNA'
job_id = jobid()
db_connector = MongoDbConnector(client=MongoClient(uri), database_id="service_requests")

Smoldyn is not properly installed in this environment and thus its process implementation cannot be registered. Please consult smoldyn documentation.
Cannot register SimpleNeuron. Error:
**
No module named 'pyneuroml'
**


In [2]:
supervisor = Supervisor(db_connector=db_connector)

In [3]:
supervisor.jobs

{'completed_jobs': [],
 'in_progress_jobs': [],
 'pending_jobs': ['c76417fe-65ed-4dc0-a680-4ec9c6fed08a',
  'a9e4f672-7dae-4e81-b777-6f7c43ba61e5']}

In [4]:
supervisor_response = supervisor.initialize()

In [5]:
supervisor_response

In [6]:
supervisor_response

In [7]:
supervisor.jobs

{'completed_jobs': [],
 'in_progress_jobs': [],
 'pending_jobs': ['33a977f1-595a-47ff-aa3e-e132cd629b87',
  'e097bfdf-ee21-4d6e-baea-28829f3140b5']}

In [8]:
[v for v in supervisor.db_connector.db.completed_jobs.find()]

[]

In [4]:
job_id = supervisor.jobs['pending_jobs'][1]

In [5]:
job_id

'a9e4f672-7dae-4e81-b777-6f7c43ba61e5'

In [6]:
job_doc = supervisor.db_connector.db.pending_jobs.find_one({'job_id': job_id})

In [7]:
job_doc

{'_id': ObjectId('6677a9d6a9d639de97d9cd01'),
 'status': 'PENDING',
 'job_id': 'a9e4f672-7dae-4e81-b777-6f7c43ba61e5',
 'omex_path': './tmp/Elowitz-Nature-2000-Repressilator.omex',
 'simulators': ['amici', 'tellurium'],
 'comparison_id': 'test-1',
 'timestamp': '2024-06-23 04:51:34.015620',
 'ground_truth_report_path': './tmp/reports.h5',
 'include_outputs': False}

In [8]:
job_params = job_doc.copy()

In [9]:
list(map(lambda k: job_params.pop(k), ['job_id', 'status', 'timestamp', '_id']))

['a9e4f672-7dae-4e81-b777-6f7c43ba61e5',
 'PENDING',
 '2024-06-23 04:51:34.015620',
 ObjectId('6677a9d6a9d639de97d9cd01')]

In [10]:
job_params['omex_path'] = job_params['omex_path'].replace('./', '../')

In [11]:
job_params['ground_truth_report_path'] = job_params['ground_truth_report_path'].replace('./', '../')

In [12]:
import os 

os.path.exists(job_params['omex_path'])

True

In [13]:
worker = Worker(job_params=job_params)

TypeError: 'BiosimulationsRunOutputData' object is not iterable

In [16]:
from verification_service import unique_id


completed_doc = supervisor.db_connector.insert_completed_job(
    job_id=unique_id(),
    comparison_id="test-0",
    results=worker.job_result
)

NameError: name 'worker' is not defined